In [237]:
import numpy as np
from astropy.table import Table
from astropy import table
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.io.votable import parse
from astropy.constants import G, M_sun, pc, R_sun, h, c, k_B
from dust_extinction.parameter_averages import G23
import astropy.units as u
from scipy.optimize import minimize
import WD_models
import os.path
from os import listdir
import matplotlib.pyplot as plt
from csv import writer
from tqdm import tqdm
%matplotlib tk

# Fitting SEDs

## Functions

Global variables:
* allwise_zp_table
* synt_table
* gaia_galex_zp_table
* bands_table

Functions:
* get_allwise_photometry(source_table)
* get_gaia_galex_photometry(source_table)
* get_synthetic_photometry(source_table)
* get_photometry(source_table)

* get_distance(parallax, parallax_err)
    * get parallax in mas, return distance in m
* blackbody_flux(wavelength, temperature)
    * get wavelength and temperature with astropy units 
    * return flux density in erg/s/cm^2/AA at the star surface
* blackbody_mod_table(teff,radius,parallax,parallax_err)
    * get teff in K (no astropy units), radius in solar radius (no astropy units), parallax, parallax_err
    * apply inverse square law to get flux at Earth
    * return a table with blackbody model fluxes in all bands (erg/s/cm^2/AA)
* deredden_obs_table(tbl)
    * get the output of "get_photometry": a table with observed fluxes in all bands (erg/s/cm^2/AA), parallax, extinction.
    * apply extinction correction.
    * return a table with dereddened fluxes in all bands (erg/s/cm^2/AA).
* get_chi2(obs_table_dered,mod_table)
    * get the output of "deredden_obs_table" and "blackbody_mod_table"
    * return the chi2 between observed and model fluxes
* plot_obs_sed(sources,idx,lam_f_lam=False,model=None)
    * get photometry, and deredden the "idx"-th entry of "sources"
    * plot the observed SED
    * set "lam_f_lam" True to plot lambda*F_lambda instead of F_lambda
    * set "model" to a tuple (teff,radius) to plot a blackbody model

In [173]:
## filter zero points, effective wavelengths, column names

## f = f0 * 10^(-0.4*(m-m0)) (erg/s/cm^2/AA) 

## wise_f0 = (np.array([309.540,171.787,31.674]) * u.Jy.to(u.erg/u.s/u.cm**2/u.Hz) * u.erg/u.s/u.cm**2/u.Hz * c.cgs / (np.array([33526,46030,115608]) * u.AA)**2).to(u.erg/u.s/u.cm**2/u.AA)

## gaia_f0 = np.array([3.009167e-21,1.346109e-21,1.638483e-21]) * (u.W/u.m**2/u.nm).to(u.erg/u.s/u.cm**2/u.AA)

# two_mass_f0 = np.array([3.129e-13,1.133e-13,4.283e-14]) * (u.W / u.cm**2 / u.micron).to(u.erg/u.s/u.cm**2/u.AA)

allwise_zp_table = Table({'band': ['2MASS.J','2MASS.H', '2MASS.Ks', 'WISE.W1', 'WISE.W2', 'WISE.W3'],
                  'col':['Jmag','Hmag','Kmag','W1mag','W2mag','W3mag'],
                  'lambda_eff':[12350,16620,21590,33526,46030,115608],
                  'f0':[3.129e-10,1.133e-10,4.283e-11,8.256082e-12,2.4306871e-12,7.1047343e-14],
                  'm0':[0,0,0,0,0,0]})

gaia_galex_zp_table = Table({'band': ['GALEX.FUV', 'GALEX.NUV','GAIA3.Gbp','GAIA3.G','GAIA3.Grp'],
                       'col':['fuv_mag','nuv_mag','phot_bp_mean_mag','phot_g_mean_mag','phot_rp_mean_mag'],
                       'err_col':['fuv_magerr','nuv_magerr','phot_bp_mean_flux_error','phot_g_mean_flux_error','phot_rp_mean_flux_error'],
                       'lambda_eff':[1516,2267,5180,6390,7820],
                       'f0':[1.4e-15,2.06e-16,3.009167e-19,1.346109e-19,1.638483e-19],
                       'm0':[18.82,20.08,25.3540,25.8010,25.1040]})

synt_table = Table({'band': ['Johnson.U', 'Johnson.B','Johnson.V','Johnson.R','Johnson.I',
                             'SDSS.u','SDSS.g','SDSS.r','SDSS.i','SDSS.z'],
                     'lambda_eff':[3551.05,4369.53,5467.57,6695.83,8568.89,3608.04,4671.78,6141.12,7457.89,8922.78],
                    'col':['u_jkc_flux','b_jkc_flux','v_jkc_flux','r_jkc_flux','i_jkc_flux',
                           'u_sdss_flux','g_sdss_flux','r_sdss_flux','i_sdss_flux','z_sdss_flux']})
bands_table = Table({'band': list(gaia_galex_zp_table['band'])+list(allwise_zp_table['band'])+list(synt_table['band']),
                     'lambda_eff': list(gaia_galex_zp_table['lambda_eff'])+list(allwise_zp_table['lambda_eff'])+list(synt_table['lambda_eff'])})

Table.sort(bands_table, 'lambda_eff')

In [121]:
## get photometry

def get_synthetic_photometry(source_table):
    id_lst = tuple(source_table['source_id'])
    cols_to_query =','.join(['source_id'] + [c + ',' + c +'_error' for c in synt_table['col']])
    query = 'SELECT ' + cols_to_query + f' FROM gaiadr3.synthetic_photometry_gspc WHERE source_id IN {id_lst}'
    job = Gaia.launch_job(query)
    result = job.get_results()
    for col in synt_table['col']:
        band = synt_table[synt_table['col']==col]['band'][0]
        err_col = col + '_error'
        if band.startswith('Johnson'): ## Johnson fluxes given in W/s/m^2/nm
            result[col] = result[col].to(u.erg/u.s/u.cm**2/u.AA)
            result[err_col] = result[err_col].to(u.erg/u.s/u.cm**2/u.AA)
        elif band.startswith('SDSS'): ## SDSS fluxes given in W/s/m^2/Hz
            wl = synt_table[synt_table['col']==col]['lambda_eff'][0] * u.AA
            result[col] = (result[col].data * u.W / u.m**2 / u.Hz * c / wl.si**2).to(u.erg/u.s/u.cm**2/u.AA)
            result[err_col] = (result[err_col].data * u.W / u.m**2 / u.Hz * c / wl.si**2).to(u.erg/u.s/u.cm**2/u.AA)
        result[col].name = band
        result[err_col].name = band +'_err'
    return result

def get_allwise_photometry(source_table):
    coords = SkyCoord(source_table['ra'], source_table['dec'], unit=(u.deg, u.deg), frame='icrs')
    result = Vizier.query_region(coords, radius=1*u.arcsec,catalog=['II/328'])
    result = result[0]
    result.sort('_q')
    for col in allwise_zp_table['col']:
        band = allwise_zp_table[allwise_zp_table['col']==col]['band'][0]
        f0 = allwise_zp_table[allwise_zp_table['col']==col]['f0'][0]
        m0 = allwise_zp_table[allwise_zp_table['col']==col]['m0'][0]
        result[col] = f0 * 10**(-0.4*(result[col]-m0))
        result[col].unit = u.erg / u.s / u.cm**2 / u.AA
        result[col].name = band
        err_col = 'e_' + col
        result[err_col] = result[band] * 0.4 * np.log(10) * result[err_col]
        result[err_col].unit = u.erg / u.s / u.cm**2 / u.AA
        result[err_col].name = band + '_err'
    result.keep_columns(list(allwise_zp_table['band']) + [band + '_err' for band in allwise_zp_table['band']])
    result['source_id'] = source_table['source_id']
    return result

def get_gaia_galex_photometry(source_table):
    result = source_table.copy()
    for col in gaia_galex_zp_table['col']:
        band = gaia_galex_zp_table[gaia_galex_zp_table['col']==col]['band'][0]
        err_col = gaia_galex_zp_table[gaia_galex_zp_table['col']==col]['err_col'][0]
        f0 = gaia_galex_zp_table[gaia_galex_zp_table['col']==col]['f0'][0]
        m0 = gaia_galex_zp_table[gaia_galex_zp_table['col']==col]['m0'][0]
        result[col] = f0 * 10**(-0.4*(result[col]-m0)) 
        result[col].unit = u.erg / u.s / u.cm**2 / u.AA
        result[col].name = band
        if err_col.startswith('phot'):
            result[err_col] = result[err_col] * f0 
        else:
            result[err_col] = result[band] * 0.4 * np.log(10) * result[err_col]
        result[err_col].unit = u.erg / u.s / u.cm**2 / u.AA
        result[err_col].name = band + '_err'
    result.keep_columns(['source_id'] + list(gaia_galex_zp_table['band']) + [band + '_err' for band in gaia_galex_zp_table['band']])
    return result

def get_photometry(source_table):
    tbl = source_table['source_id','ra','dec','parallax','parallax_error','mh_for_mass_interp','av_for_mass_interp']
    tbl = table.join(tbl, get_gaia_galex_photometry(source_table), keys='source_id', join_type='left')
    tbl = table.join(tbl, get_synthetic_photometry(tbl), keys='source_id', join_type='left')
    tbl = table.join(tbl, get_allwise_photometry(tbl), keys='source_id', join_type='left')
    return tbl

In [262]:
## model and fitting functions

filter_list = ['Obs.Flux_2MASS.H', 'Obs.Flux_2MASS.J', 'Obs.Flux_2MASS.Ks',
                'Obs.Flux_ACS_WFC.F606W', 'Obs.Flux_ACS_WFC.F814W',
                  'Obs.Flux_APASS.B', 'Obs.Flux_APASS.V',
                    'Obs.Flux_DECam.Y', 'Obs.Flux_DECam.g', 'Obs.Flux_DECam.i', 'Obs.Flux_DECam.r', 'Obs.Flux_DECam.z',
                      'Obs.Flux_DENIS.I', 'Obs.Flux_DENIS.J', 'Obs.Flux_DENIS.Ks',
                        'Obs.Flux_GAIA3.G', 'Obs.Flux_GAIA3.Gbp', 'Obs.Flux_GAIA3.Grp', 'Obs.Flux_GAIA3.Grvs',
                          'Obs.Flux_GALEX.FUV', 'Obs.Flux_GALEX.NUV',
                            'Obs.Flux_Johnson.B', 'Obs.Flux_Johnson.I', 'Obs.Flux_Johnson.R', 'Obs.Flux_Johnson.U', 'Obs.Flux_Johnson.V',
                              'Obs.Flux_PS1.g', 'Obs.Flux_PS1.i', 'Obs.Flux_PS1.r', 'Obs.Flux_PS1.y', 'Obs.Flux_PS1.z',
                                'Obs.Flux_SDSS.g', 'Obs.Flux_SDSS.i', 'Obs.Flux_SDSS.r', 'Obs.Flux_SDSS.u', 'Obs.Flux_SDSS.z',
                                  'Obs.Flux_UKIDSS.K',
                                    'Obs.Flux_VISTA.H', 'Obs.Flux_VISTA.J', 'Obs.Flux_VISTA.Ks', 'Obs.Flux_VISTA.Y',
                                      'Obs.Flux_WISE.W1', 'Obs.Flux_WISE.W2', 'Obs.Flux_WISE.W3']

def get_distance(parallax,parallax_err):
    ## distance in meters calculated from parallax in mas
    return (1000/parallax) * pc.value , (1000/parallax**2)*parallax_err * pc.value

def blackbody_flux(wavelength, temperature):
    ## wavelength and temperature must have astropy units
    f = (2 * np.pi * 2 * h.cgs * c.cgs**2 / wavelength.cgs**5) / (np.exp(h.cgs * c.cgs / (wavelength.cgs * k_B.cgs * temperature)) - 1)
    return f.to(u.erg / u.s / u.cm**2 / u.AA)

def blackbody_mod_table(teff,radius,parallax,parallax_err):
    bands = list(bands_table['band'])
    mod_tbl = Table(dict(zip(bands,[[float(1)] for i in range(len(bands))])))
    d, d_err = get_distance(parallax,parallax_err)
    radius = radius * R_sun.value
    for b in bands:
        filepath = os.path.join('..','data','VOSA','filters',b+'.dat')
        filter_tbl = Table.read(filepath, format='ascii', names=['wavelength','transmission'])
        wavelength = filter_tbl['wavelength'].data * u.AA
        
        transmission = filter_tbl['transmission'].data 
        flux = blackbody_flux(wavelength, teff * u.K)
        flux = np.dot(flux, transmission)
        flux /= np.sum(transmission)
        flux = flux * (radius/d)**2
        mod_tbl[0][b] = flux.value
        mod_tbl[b].unit = flux.unit
    return mod_tbl
        
def deredden_obs_table(tbl):
    tbl_dered = tbl.copy()
    ext = G23(Rv=3.1)
    cols = list(bands_table['band'])
    wavelengths = list(bands_table['lambda_eff'])
    for col, wavelength in zip(cols, wavelengths):
      tbl_dered[col] = tbl[col] /ext.extinguish(wavelength*u.AA, Av=tbl['av_for_mass_interp'].data)
    return tbl_dered

def get_chi2(obs_tbl_dered, mod_tbl, no_uv = False):
    chi2 = 0
    for col in mod_tbl.colnames:
        if no_uv and col.startswith('GALEX'):
           continue
        if col in obs_tbl_dered.colnames:
            if not(np.ma.is_masked(obs_tbl_dered[col]) or np.ma.is_masked(obs_tbl_dered[col + '_err'])):
                chi2 += np.sum((obs_tbl_dered[col] - mod_tbl[col])**2 /(0.1 * obs_tbl_dered[col])**2)
    return chi2

def plot_obs_sed(sources,idx,lam_f_lam=False,model=None):
    obs_tbl = get_photometry(sources)
    obs_tbl_dered = deredden_obs_table(obs_tbl)

    bnds = list(bands_table['band'])
    wl = np.array(list(bands_table['lambda_eff']))
    flux = np.array([obs_tbl_dered[idx][bnd] for bnd in bnds])
    flux_err = 0.1 * flux

    fig,ax = plt.subplots(dpi = 120,)
    if lam_f_lam:
      ax.errorbar(wl,wl * flux,yerr = wl * flux_err,fmt='.',label='observed',ecolor='k',elinewidth=1,mec='Crimson',mfc='Crimson',capsize=3)
      ax.set_ylabel(r'$\lambda f_\lambda $($erg/s/cm^2/\AA$)',fontsize=16)
    else:
      ax.errorbar(wl,flux,yerr = flux_err,fmt='.',label='observed',ecolor='Salmon',elinewidth=1,mec='Crimson',mfc='Crimson',capsize=3)
      ax.set_ylabel(r'$f_\lambda $($erg/s/cm^2/\AA$)',fontsize=16)
    if model is not None:
        teff, radius = model
        mod_tbl = blackbody_mod_table(teff,radius,obs_tbl[idx]['parallax'],obs_tbl[idx]['parallax_error'])
        mod_flux = np.array([mod_tbl[bnd][0] for bnd in bnds])
        if lam_f_lam:
          ax.plot(wl,wl * mod_flux,color='RoyalBlue',ls='--',label=f'Blackbody {int(teff)}K/{radius:.2f} $R_\odot$',marker='.',mec='w',mfc='w')
        else:
          ax.plot(wl,mod_flux,color='RoyalBlue',ls='--',label=f'Blackbody {int(teff)}K/{radius:.2f} $R_\odot$',marker='.',mec='w',mfc='w')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'wavelength $(\AA)$',fontsize=16)
    ax.set_title('SED',fontsize=16)
    ax.set_facecolor('0.25')
    ax.tick_params(axis='both', which='major', labelsize=16)
    ax.grid()
    ax.legend(fontsize=16)
    plt.show()

def fit_table(sources, init = (6000,1) ,teff_bounds = (3000,10000), r_bounds = (0.4,3.0), no_uv = True):
    ## init, teff_bounds, r_bounds either a tuple or a list of tuples of length len(sources)
    ## if list of tuples, each tuple is for a different source
    obs_tbl = get_photometry(sources)
    obs_tbl_dered = deredden_obs_table(obs_tbl)
    results = np.zeros((len(sources),2))
    
    if type(init) == tuple:
      init = [init for i in range(len(sources))]
    elif len(init) != len(sources):
      raise ValueError('init must be a tuple or a list of tuples of length len(sources)')
    if type(teff_bounds) == tuple:
      teff_bounds = [teff_bounds for i in range(len(sources))]
    elif len(teff_bounds) != len(sources):
      raise ValueError('teff_bounds must be a tuple or a list of tuples of length len(sources)')
    if type(r_bounds) == tuple:
      r_bounds = [r_bounds for i in range(len(sources))]
    elif len(r_bounds) != len(sources):
      raise ValueError('r_bounds must be a tuple or a list of tuples of length len(sources)')

    def fun(params,obs_row):
      teff, radius = params
      mod_tbl = blackbody_mod_table(teff,radius,obs_row['parallax'],obs_row['parallax_error'])
      return get_chi2(obs_row, mod_tbl, no_uv)
    
    i = 0
    for obs_row,init,teff_bounds,r_bounds in tqdm(zip(obs_tbl_dered,init,teff_bounds,r_bounds),total=len(sources)):
      result = minimize(fun, list(init),args = (obs_row), bounds = (teff_bounds,r_bounds), options={'maxiter':100})
      teff, radius = result.x
      plot_obs_sed(sources,i,lam_f_lam=True,model=(teff,radius))
      results[i,0] = teff
      results[i,1] = radius 
      i += 1
    return results

In [263]:
# from scipy.optimize import minimize

# sources = Table.read('../table_hst.fits', format='fits')
# obs_tbl = get_photometry(sources)
# obs_tbl_dered = deredden_obs_table(obs_tbl)

# def fun(x,obs_row):
#     ## get x = (teff, radius)
#     ## return chi2
#     teff, radius = x
#     mod_tbl = blackbody_mod_table(teff,radius,obs_row['parallax'],obs_row['parallax_error'])    
#     return get_chi2(obs_row, mod_tbl, no_uv = True)

# minimize(fun, [8000,1], args=(obs_tbl_dered[1]),bounds = ((5000,10000),(0.5,2)),options={'maxiter':100})
fit_table(sources, init = (6000,1) ,teff_bounds = (3000,10000), r_bounds = (0.4,3.0), no_uv = True)

  0%|          | 0/6 [00:00<?, ?it/s]WARNING: MergeConflictWarning: Cannot merge meta key 'ucd' types <class 'str'> and <class 'str'>, choosing ucd='meta.id;meta.main' [astropy.utils.metadata]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_12644\526012752.py:70: UserWarning: Warning: converting a masked element to nan.
  flux = np.array([obs_tbl_dered[idx][bnd] for bnd in bnds])
 17%|█▋        | 1/6 [00:40<03:24, 40.89s/it]WARNING: MergeConflictWarning: Cannot merge meta key 'ucd' types <class 'str'> and <class 'str'>, choosing ucd='meta.id;meta.main' [astropy.utils.metadata]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_12644\526012752.py:70: UserWarning: Warning: converting a masked element to nan.
  flux = np.array([obs_tbl_dered[idx][bnd] for bnd in bnds])
 33%|███▎      | 2/6 [01:47<03:43, 55.84s/it]WARNING: MergeConflictWarning: Cannot merge meta key 'ucd' types <class 'str'> and <class 'str'>, choosing ucd='meta.id;meta.main' [astropy.utils.metadata]
C:\Users\ASUS\AppData\Local\Temp\ip

array([[1.00000000e+04, 1.87396970e+00],
       [7.48276725e+03, 1.13939170e+00],
       [6.14186554e+03, 1.69670451e+00],
       [5.07649921e+03, 6.58736585e-01],
       [6.24397793e+03, 1.28684167e+00],
       [8.03941018e+03, 1.48187174e+00]])

In [236]:
plot_obs_sed(sources,1,lam_f_lam=True,model=[7483,1.139])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12644\106394461.py:70: UserWarning: Warning: converting a masked element to nan.
  flux = np.array([obs_tbl_dered[idx][bnd] for bnd in bnds])


# Summing different models

In [47]:
model = WD_models.load_model(low_mass_model='Bedard2020',
                             middle_mass_model='Bedard2020',
                             high_mass_model='ONe',
                             atm_type='H')

sources = Table.read('../table_hst.fits',format='fits')

In [44]:
ms_sed_logg_arr = np.arange(0,5.1,0.5)
wd_sed_logg_arr = np.arange(6.5,9.7,0.25)
meta_sed_arr = np.array([-2.5,2.0,-1.5,-1.0,-0.5,0.0,0.2,0.5])
ms_sed_teff_arr = np.hstack([np.arange(3500,13000,250),np.arange(13000,50001,1000)])
wd_sed_teff_arr = np.hstack([np.arange(5000,20000,250),np.arange(20000,30000,1000),np.arange(30000,40000,2000),np.arange(40000,50000,5000),np.arange(50000,80001,10000)])


def get_kurucz_sed_filepath(teff, logg, alpha, meta):
    ## 3500 <= teff <= 13000 with jumps of 250 (additional templates up to 50,000, but less dense)
    ## 0.0 <= logg <= 5.0 with jumps of 0.5
    ## alpha = 0 or alpha = 0.4
    ## -4.0 <= metallicity <= 0.5 evaluated at: -4.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5

    logg = ms_sed_logg_arr[np.argmin(np.abs(ms_sed_logg_arr - logg))]
    teff = ms_sed_teff_arr[np.argmin(np.abs(ms_sed_teff_arr - teff))]
    meta = meta_sed_arr[np.argmin(np.abs(meta_sed_arr - meta))]

    dirname = os.path.join('..','data','VOSA','models_MS','kurucz')
    filename = 'Kurucz2003all_f'
    if meta >= 0:
        filename += 'p'
    elif meta <= 0:
        filename += 'm'
    meta_dig1 = int((abs(meta)*10)//10)
    meta_dig2 = int((abs(meta)*10)%10)
    filename += f'{meta_dig1}{meta_dig2}'
    if alpha > 0:
        filename += 'a'
    filename += f't{int(teff)}'
    logg_dig1 = int((logg*10)//10)
    logg_dig2 = int((logg*10)%10)
    filename += f'g{logg_dig1}{logg_dig2}'
    filename += 'k2odfnew.fl.phot.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    elif os.path.exists(path.replace('all','')):
        return path.replace('all','')
    else:
        print('no such template ' + filename)
        return ''
    
def get_koester_sed_filepath(teff, logg):
    ## 5,000 <= teff <= 50,000 with jumps of 250
    ## 6.5 <= logg <= 9.0 with jumps of 0.25

    logg = wd_sed_logg_arr[np.argmin(np.abs(wd_sed_logg_arr - logg))]
    teff = wd_sed_teff_arr[np.argmin(np.abs(wd_sed_teff_arr - teff))]
    
    dirname = os.path.join('..','data','VOSA','models_WD','koester')
    filename = 'koester2_da'
    teff_dig1 = int((teff//10000)%10)
    teff_dig2 = int((teff//1000)%10)
    teff_dig3 = int((teff//100)%10)
    teff_dig4 = int((teff//10)%10)
    teff_dig5 = int((teff//1)%10)
    filename += f'{teff_dig1}{teff_dig2}{teff_dig3}{teff_dig4}{teff_dig5}_'
    filename += f'{int(logg*100)}'
    filename += '.dk.phot.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    else:
        # print('no such template ' + filename)
        return ''
    
def get_radius(logg, m, m_err):
    ## calculate radius (in meters) from logg and the stellar mass (in units of M_sun)
    ## note to self- log g is defined as log_10 of g in cm/s^2
    r = np.sqrt(G.value * m * M_sun.value / (10**logg / 100))
    dr = r / (2*m) * m_err 
    return r , dr

def inverse_square_coeff(logg,m,m_err,parallax,parallax_err):
    r,dr = get_radius(logg,m,m_err)
    d,d_err = get_distance(parallax,parallax_err)
    c = (r/d)**2
    dc = c * np.sqrt((2*dr/r)**2 + (2*d_err/d)**2)
    return c,dc

def get_ms_logg(m, m_err):
    ## note to self- log g is defined as log_10 of g in cm/s^2

    ## use main sequence scaling relation: mass proportional to radius
    r = m * R_sun
    dr = m_err * R_sun
    g = G.value * m * M_sun.value / r**2 * 1e2 ## in cm/s^2
    dg = g * np.sqrt((2*dr/r)**2 + (2*m_err/m)**2)
    return np.log10(g.value)

def get_wd_logg(m,teff):
    ## WD radius nearly independent of mass
    m_logteff_to_logg = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                  y=model['logteff'],
                                                  z=model['logg'],
                                                  interp_type='linear')
    logg = float(m_logteff_to_logg(m,np.log10(teff)))
    return logg

def model_sed_ms(teff1, m1, m1_err, parallax, parallax_err, meta1 = -0.5, alpha1 = 0, r1 = 0):
    ## teff and logg are the parameters we wanna optimize (maybe alpha and meta too, one day)
    ## m (mass) and parallax should be given, to be used in the flux calculation
    if r1 == 0:
        logg1 = get_ms_logg(m1,m1_err)
    else:
        logg1 = np.log10(G.value * m1 * M_sun.value / (r1 * R_sun.value)**2 * 1e2)
    path1 = get_kurucz_sed_filepath(teff1, logg1, alpha1, meta1)
    if len(path1) == 0: ## wrong path: change directory path / check the parameters indeed have a template
        return Table({'filter':[], 'wavelength':[],'flux':[],'flux_err':[]}) 
    c1,dc1 = inverse_square_coeff(logg1,m1,m1_err,parallax,parallax_err)
    votable1 = parse(path1)
    table1 = Table.read(votable1.get_first_table())
    model_flux = c1 * table1['flux'].data * table1['wavelength'].data
    model_flux_err = dc1 * table1['flux'].data
    model_table = Table({'filter':table1['filter'].data, 'wavelength':table1['wavelength'].data,'flux':model_flux,'flux_err':model_flux_err})
    return model_table

def model_sed_wd(teff2,m2,m2_err,parallax,parallax_err):
      logg2 = get_wd_logg(m2,teff2)
      path2 = get_koester_sed_filepath(teff2, logg2)
      if len(path2) == 0: ## wrong path: change directory path / check the parameters indeed have a template
          return Table({'filter':[], 'wavelength':[],'flux':[],'flux_err':[]}) 
      c2,dc2 = inverse_square_coeff(logg2,m2,m2_err,parallax,parallax_err)
      votable2 = parse(path2)
      table2 = Table.read(votable2.get_first_table())
      model_flux = c2 * table2['flux'].data * table2['wavelength'].data
      model_flux_err = dc2 * table2['flux'].data
      model_table = Table({'filter':table2['filter'].data, 'wavelength':table2['wavelength'].data,'flux':model_flux,'flux_err':model_flux_err})
      return model_table

def plot_sed_ms_wd(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0):
    ms = model_sed_ms(teff1, m1, m1_err, parallax, parallax_err, meta1)
    wd = model_sed_wd(teff2, m2, m2_err, parallax, parallax_err)
    ms.sort('wavelength')
    wd.sort('wavelength')
    
    tot_flux = ms['flux'] + wd['flux']

    fig , ax = plt.subplots(figsize=(5,5),dpi = 120)
    ax.plot(ms['wavelength'],ms['flux'],linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(wd['wavelength'],wd['flux'],linestyle='--',color='Crimson',label=f'WD {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(ms['wavelength'],tot_flux,color='k',label=f'total flux',linewidth=1,marker='.',markersize=5)
    ax.scatter(ms['wavelength'][1],tot_flux[1],color='Indigo',s=50,label='Galex NUV',marker='o')
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_facecolor('0.7')
    ax.set_ylim(1e-20,10*np.max(ms['flux']))
    ax.legend()
    ax.grid()
    fig.show()

def get_separatrix_fuv(m1,m2,excess):
    parallax = 1
    meta = 0
    
    x,y = np.meshgrid(ms_sed_teff_arr[ms_sed_teff_arr < 10000],wd_sed_teff_arr)
    j = 0
    teff2_f = []
    
    for i in tqdm(range(x.shape[1])):
        fuv_excess = 0
        while fuv_excess < excess and j < x.shape[0]:
            t1 = x[j][i]
            t2 = y[j][i]
            ms = model_sed_ms(t1, m1, m1 * 0.1, parallax, parallax * 0.1, meta)
            wd = model_sed_wd(t2, m2, m2 * 0.1, parallax, parallax * 0.1)
            try:       
                ms_fuv = ms[ms['filter'] == 'GALEX/GALEX.FUV']['flux'][0]
                wd_fuv = wd[wd['filter'] == 'GALEX/GALEX.FUV']['flux'][0]
                fuv_excess = 1 + wd_fuv / ms_fuv
            except:
                fuv_excess = 0
            j += 1
        if j < x.shape[0]:
            teff2_f.append(t2)
    return x[0,0:len(teff2_f)], np.array(teff2_f)
    

def get_separatrix_nuv(m1,m2,excess):
    parallax = 1
    meta = 0
    
    x,y = np.meshgrid(ms_sed_teff_arr[ms_sed_teff_arr < 10000],wd_sed_teff_arr)
    j = 0
    teff2_n = []
    
    for i in tqdm(range(x.shape[1])):
        nuv_excess = 0
        while nuv_excess < excess and j < x.shape[0]:
            t1 = x[j][i]
            t2 = y[j][i]
            ms = model_sed_ms(t1, m1, m1 * 0.1, parallax, parallax * 0.1, meta, r1 = r1)
            wd = model_sed_wd(t2, m2, m2 * 0.1, parallax, parallax * 0.1)
            try:       
                ms_nuv = ms[ms['filter'] == 'GALEX/GALEX.NUV']['flux'][0]
                wd_nuv = wd[wd['filter'] == 'GALEX/GALEX.NUV']['flux'][0]
                nuv_excess = 1 + wd_nuv / ms_nuv
            except:
                nuv_excess = 0
            j += 1
        if j < x.shape[0]:
            teff2_n.append(t2)
    return x[0,0:len(teff2_n)], np.array(teff2_n)

def plot_separatrix_nuv(excess,m1,m1_err,m2,m2_err,teff1,teff2):
    ## excess is: total flux / single star flux

    r1 = get_radius(get_ms_logg(m1,m1_err),m1,m1_err)[0] / R_sun.value
    r2 = get_radius(get_wd_logg(m2,teff2),m2,m2_err)[0] / R_sun.value
    x,y = get_separatrix_nuv(m1,m2,excess)
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_xlabel(r'$T_{eff,MS}$')
    ax.set_ylabel(r'$T_{eff,WD}$')    
    textstr = '\n'.join((f'$M_1 = {m1:.2f} M_\odot$' + f'$R_1 = {r1:.2f} R_\odot$',
                        f'$M_2 = {m2:.2f} M_\odot$' + f'$R_2 = {r2:.2f} R_\odot$'))
    props = dict(boxstyle='round', facecolor='DimGray',edgecolor='k',alpha = 0.8)
    ax.text(0.02, 0.8, textstr, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=props)
    ax.plot(x,y,label=f'{int((excess - 1)*100)}% NUV excess')
    
    ax.scatter(teff1,teff2,label='Current estimate',color='Crimson',marker='x',s=50)
    ax.legend(loc = [0.02,0.84],facecolor='DimGray',edgecolor='k')
    ax.grid()
    plt.show()

In [45]:
ms_logg_arr = np.array([3.5,4.0,4.5,5.0])
wd_logg_arr = np.arange(6.5,9.7,0.25)
meta_arr = np.array([-2.5,2.0,-1.5,-1.0,-0.5,0.0,0.2,0.5])
ms_teff_arr = np.hstack([np.arange(3500,13000,250),np.arange(13000,50001,1000)])
wd_teff_arr = np.hstack([np.arange(5000,20000,250),np.arange(20000,30000,1000),np.arange(30000,40000,2000),np.arange(40000,50000,5000),np.arange(50000,80000,10000)])

def get_kurucz_spec_filepath(teff, logg, meta):
    ## 3500 <= teff <= 50000 with jumps of 250 up to 13,000
    ## 3.5 <= logg <= 5.0 with jumps of 0.5
    ## -2.5 <= metallicity <= 0.5 evaluated at: -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5
    dirname = os.path.join('..','data','VOSA','spectra_MS','Kurucz')
    filename = 'f'

    logg = ms_logg_arr[np.argmin(np.abs(ms_logg_arr - logg))]
    meta = meta_arr[np.argmin(np.abs(meta_arr - meta))]
    teff = ms_teff_arr[np.argmin(np.abs(ms_teff_arr - teff))]

    if meta >= 0:
        filename += 'p'
    elif meta <= 0:
        filename += 'm'
    meta_dig1 = int((abs(meta)*10)//10)
    meta_dig2 = int((abs(meta)*10)%10)
    filename += f'{meta_dig1}{meta_dig2}'
    filename += 'k2odfnew.pck.teff='
    filename += f'{int(teff)}..logg='
    filename += f'{logg:0.5f}'
    filename += '.dat.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    else:
        print('no such template ' + filename)
        return ''
    
def get_koester_spec_filepath(teff, logg):
    ## 5,000 <= teff <= 80,000 with jumps of 250
    ## 7.0 <= logg <= 8.0 with jumps of 0.25
    dirname = os.path.join('..','data','VOSA','spectra_WD','Koester')
    filename = 'da'
    
    logg = wd_logg_arr[np.argmin(np.abs(wd_logg_arr - logg))]
    teff = wd_teff_arr[np.argmin(np.abs(wd_teff_arr - teff))]

    teff_dig1 = int((teff//10000)%10)
    teff_dig2 = int((teff//1000)%10)
    teff_dig3 = int((teff//100)%10)
    teff_dig4 = int((teff//10)%10)
    teff_dig5 = int((teff//1)%10)
    filename += f'{teff_dig1}{teff_dig2}{teff_dig3}{teff_dig4}{teff_dig5}_'
    filename += f'{int(logg*100)}'
    filename += '.dk.dat.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    else:
        print('no such template ' + filename)
        return ''

def get_ms_logg(m, m_err):
    ## note to self- log g is defined as log_10 of g in cm/s^2

    ## use main sequence scaling relation: mass proportional to radius
    r = m * R_sun
    dr = m_err * R_sun
    g = G.value * m * M_sun.value / r**2 * 1e2 ## in cm/s^2
    dg = g * np.sqrt((2*dr/r)**2 + (2*m_err/m)**2)
    return np.log10(g.value) 

def get_wd_logg(m,teff):
    ## WD radius nearly independent of mass
    m_logteff_to_logg = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                  y=model['logteff'],
                                                  z=model['logg'],
                                                  interp_type='linear')
    logg = float(m_logteff_to_logg(m,np.log10(teff)))
    return logg

def get_distance(parallax,parallax_err):
    ## distance in meters calculated from parallax in mas
    return (1000/parallax) * pc.value , (1000/parallax**2)*parallax_err * pc.value

def inverse_square_coeff(logg, m, m_err, parallax, parallax_err):
    R , dR = get_radius(logg, m, m_err)
    D , dD = get_distance(parallax, parallax_err)
    c = (R / D)**2
    dc = np.sqrt((2*(R/D**2)*dR)**2 + (2*(R**2/D**3)*dD)**2)
    return c , dc

def model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1 = -0.5):
    ## teff and logg are the parameters we wanna optimize (maybe alpha and meta too, one day)
    ## m (mass) and parallax should be given, to be used in the flux calculation
    logg1 = get_ms_logg(m1,m1_err)
    path1 = get_kurucz_spec_filepath(teff1, logg1, meta1)
    if len(path1) == 0: ## wrong path: change directory path / check the parameters indeed have a template
        return Table({'wavelength':[],'wavelength':[]}) 
    c1,dc1 = inverse_square_coeff(logg1,m1,m1_err,parallax,parallax_err)
    votable1 = parse(path1)
    table1 = Table.read(votable1.get_first_table())
    model_flux = c1 * table1['FLUX'].data * table1['WAVELENGTH'].data
    model_table = Table({'wavelength':table1['WAVELENGTH'].data,'flux':model_flux})
    return model_table

def model_WD(teff2, m2, m2_err, parallax, parallax_err):
    ## teff and logg are the parameters we wanna optimize
    ## m (mass) and parallax should be given, to be used in the flux calculation
    logg2 = get_wd_logg(m2,teff2)
    path2 = get_koester_spec_filepath(teff2, logg2)
    if len(path2) == 0: ## wrong path: change directory path / check the parameters indeed have a template
        return Table({'wavelength':[],'flux':[]}) 
    c2,dc2 = inverse_square_coeff(logg2,m2,m2_err,parallax,parallax_err)
    votable2 = parse(path2)
    table2 = Table.read(votable2.get_first_table())
    model_flux = c2 * table2['FLUX'].data * table2['WAVELENGTH'].data
    model_table = Table({'wavelength':table2['WAVELENGTH'].data,'flux':model_flux})
    return model_table

def plot_binary_ms(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0, meta2  = 0):
    ms1 = model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1)
    ms2 = model_MS(teff2, m2, m2_err, parallax, parallax_err, meta2)
    ms1.sort('wavelength')
    ms2.sort('wavelength')

    shared_wl = np.union1d(ms1['wavelength'].data,ms2['wavelength'].data)
    m1_interp = np.interp(shared_wl,ms1['wavelength'].data,ms1['flux'].data, left = 0, right = 0)
    m2_interp = np.interp(shared_wl,ms2['wavelength'].data,ms2['flux'].data, left = 0, right = 0)
    
    tot_flux = m1_interp + m2_interp
    
    fig , ax = plt.subplots(figsize=(5,5),dpi = 120)
    ax.plot(ms1['wavelength'],ms1['flux'],linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(ms2['wavelength'],ms2['flux'],linestyle='--',color='Crimson',label=f'MS {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(shared_wl,tot_flux,color='k',label=f'total flux',linewidth=1)
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_ylim(1e-20,10*np.max(ms1['flux']))
    ax.set_facecolor('0.7')
    ax.legend()
    ax.grid()
    fig.show()

def plot_triple_ms(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0, meta2  = 0):
    ms1 = model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1)
    ms2 = model_MS(teff2, m2 / 2, m2_err, parallax, parallax_err, meta2)
    ms1.sort('wavelength')
    ms2.sort('wavelength')

    shared_wl = np.union1d(ms1['wavelength'].data,ms2['wavelength'].data)
    m1_interp = np.interp(shared_wl,ms1['wavelength'].data,ms1['flux'].data, left = 0, right = 0)
    m23_interp = np.interp(shared_wl,ms2['wavelength'].data,ms2['flux'].data, left = 0, right = 0)
    
    tot_flux = m1_interp + 2 * m23_interp
    
    fig , ax = plt.subplots(figsize=(5,5),dpi = 120)
    ax.plot(ms1['wavelength'],ms1['flux'],linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(ms2['wavelength'],2 *ms2['flux'],linestyle='--',color='Crimson',label=f'double MS {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(m23_interp,tot_flux,color='k',label=f'total flux',linewidth=1)
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_facecolor('0.7')
    ax.set_ylim(1e-20,10*np.max(ms1['flux']))
    ax.legend()
    ax.grid()
    fig.show()

def plot_binary_wd(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0, gratings = ['galex']):
    ms = model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1)
    wd = model_WD(teff2, m2, m2_err, parallax, parallax_err)
    ms.sort('wavelength')
    wd.sort('wavelength')
    
    shared_wl = np.union1d(ms['wavelength'].data,wd['wavelength'].data)
    m_interp = np.interp(shared_wl,ms['wavelength'].data,ms['flux'].data, left = 0, right = 0)
    w_interp = np.interp(shared_wl,wd['wavelength'].data,wd['flux'].data, left = 0, right = 0)
    tot_flux = m_interp + w_interp

    fig , ax = plt.subplots(figsize=(5,5),dpi = 120)
    ax.plot(ms['wavelength'],ms['flux'] ,linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(wd['wavelength'],wd['flux'] ,linestyle='--',color='Crimson',label=f'WD {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(shared_wl,tot_flux,color='k',label=f'total flux',linewidth=1)
    if 800 in gratings:
        ax.vlines([815,1948],0,100,colors='Gold',lw=1,ls='--',label='COS G140L/800',alpha = 0.8)
    elif 1105 in gratings:
        ax.vlines([1118,2150],0,100,colors='Gold',lw=1,ls='--',label='COS G140L/1105',alpha = 0.8)
    if 2635 in gratings:
        ax.vlines([2435,2834],0,100,colors='Indigo',lw=1,ls='--',label='COS G230L/2635',alpha = 0.8)
    elif 'galex' in gratings:
        ax.vlines([1750,2800],0,100,colors='Indigo',lw=1,ls='--',label='GALEX NUV',alpha = 0.8)
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_facecolor('0.7')
    ax.set_ylim(1e-20,10*np.max(ms['flux']))
    ax.legend()
    ax.grid()
    fig.show()
    return shared_wl, tot_flux / shared_wl ## wl in angstrom, flux in erg/s/cm^2/A

In [48]:
i = 1
sources = Table.read('../table_hst.fits',format='fits')

m1 = sources[i]['m1']
m1_err = sources[i]['m1_err']
m2 = sources[i]['m2']
if m2>1.25:
    m2 = sources[i]['m2'] - sources[i]['m2_err']
m2_err = sources[i]['m2_err']

parallax = sources[i]['parallax']
parallax_err = sources[i]['parallax_error']

teff1 = sources[i]['Teff1']
teff2 = sources[i]['Teff2']

meta = sources[i]['mh_for_mass_interp']

wl, fl = plot_binary_wd(teff1,m1,m1_err,teff2,m2,m2_err,parallax,parallax_err,meta,gratings = [800])
# plot_sed_ms_wd(teff1,m1,m1_err,teff2,m2,m2_err,parallax,parallax_err,meta)
# plot_binary_ms(8000,m1,m1_err,6000,m2,m2_err,parallax,parallax_err)
# plot_triple_ms(9000,m1,m1_err,3500,m2,m2_err,parallax,parallax_err)


# tbl = Table({'wavelength':wl,'flux':fl})
# cond = (tbl['wavelength'] > 925) & (tbl['wavelength'] < 80000)
# tbl = tbl[cond]
# tbl.write('../SBA28.dat',format='ascii',overwrite=True)

In [71]:
fuv_m0 = 18.82 # AB mag
nuv_m0 = 20.08 # AB mag
fuv_f0 = 1.4e-15 # erg/s/cm^2/A
nuv_f0 = 2.06e-16 # erg/s/cm^2/A

def obs_nuv_flux(mag,av):
    ext = G23(Rv = 3.1)
    f = 10**(-0.4 * (mag - nuv_m0)) * nuv_f0
    f_dereddened = f / ext.extinguish(2303 * u.AA, Av = av)
    return f_dereddened

def get_primary_nuv_lims(t_lo,t_hi,m_lo,m_hi,r_lo,r_hi,parallax,meta):
    ## get the NUV flux range of the primary star
    ms_hi = model_sed_ms(t_hi, m_hi, m_hi*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r_hi)
    ms_lo = model_sed_ms(t_lo, m_lo, m_lo*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r_lo)
    nuv_hi = ms_hi[ms_hi['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / ms_hi[ms_hi['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
    nuv_lo = ms_lo[ms_lo['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / ms_lo[ms_lo['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
    return nuv_lo, nuv_hi

def get_companion_RT_lims(obs_mag,nuv_lo,nuv_hi,m_lo,m_hi,parallax,av):
    obs_flux = obs_nuv_flux(obs_mag,av)
    
    flux_lo = np.zeros(len(wd_sed_teff_arr))
    flux_hi = np.zeros(len(wd_sed_teff_arr))
    for i,t in enumerate(wd_sed_teff_arr):
        wd_lo = model_sed_wd(t, m_lo, m_lo*0.1, parallax, parallax*0.1)
        wd_hi = model_sed_wd(t, m_hi, m_hi*0.1, parallax, parallax*0.1)
        flux_lo[i] = wd_lo[wd_lo['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / wd_lo[wd_lo['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
        flux_hi[i] = wd_hi[wd_hi['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / wd_hi[wd_hi['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]

    excess_hi = obs_flux - flux_lo
    excess_lo = obs_flux - flux_hi
    t1 = wd_sed_teff_arr[np.argmin(np.abs(excess_lo - nuv_lo))]
    t2 = wd_sed_teff_arr[np.argmin(np.abs(excess_lo - nuv_hi))]
    t3 = wd_sed_teff_arr[np.argmin(np.abs(excess_hi - nuv_lo))]
    t4 = wd_sed_teff_arr[np.argmin(np.abs(excess_hi - nuv_hi))]
    r1 = get_radius(get_wd_logg(m_lo,t1),m_lo,m_lo*0.1)[0] / R_sun.value
    r2 = get_radius(get_wd_logg(m_lo,t2),m_lo,m_lo*0.1)[0] / R_sun.value
    r3 = get_radius(get_wd_logg(m_hi,t3),m_hi,m_hi*0.1)[0] / R_sun.value
    r4 = get_radius(get_wd_logg(m_hi,t4),m_hi,m_hi*0.1)[0] / R_sun.value
    
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_xlabel(r'$T_{eff,WD}$')
    ax.set_ylabel(r'$R_{WD}$')
    ax.fill([t1,t2,t3,t4],[r1,r2,r3,r4],color='DimGray',alpha=0.5)
    fig.show()
    return [t1,t2,t3,t4],[r1,r2,r3,r4]


In [81]:
i = 1

t_lo,t_hi = 5000,5500
m_lo,m_hi= 0.90,0.96
r_lo,r_hi = 0.84,1.0
parallax = sources[i]['parallax']
meta = sources[i]['mh_for_mass_interp']
av = sources[i]['av_for_mass_interp']
obs_mag = sources[i]['nuv_mag']

nuv_lo,nuv_hi = get_primary_nuv_lims(t_lo,t_hi,m_lo,m_hi,r_lo,r_hi,parallax,meta)
obs_flux = obs_nuv_flux(obs_mag,av)
excess_hi = obs_flux - nuv_lo    
excess_lo = obs_flux - nuv_hi
wd_lo,wd_hi = 0.45,0.5
get_companion_RT_lims(obs_mag,nuv_lo,nuv_hi,wd_lo,wd_hi,parallax,av)

([70000, 70000, 60000, 60000],
 [nan, nan, 0.022081943512562974, 0.022081943512562974])

In [7]:
# from dust_extinction.parameter_averages import G23
# import astropy.units as u

# ext = G23(Rv = 3.1)

# fuv_m0 = 18.82 # AB mag
# nuv_m0 = 20.08 # AB mag
# fuv_f0 = 1.4e-15 # erg/s/cm^2/A
# nuv_f0 = 2.06e-16 # erg/s/cm^2/A

# f_excess = []
# n_excess = []

# for i in range(len(sources)):
#     av = sources[i]['av_for_mass_interp']
#     m1 = sources[i]['m1']
#     m1_err = sources[i]['m1_err']
#     parallax = sources[i]['parallax']
#     parallax_err = sources[i]['parallax_error']
#     teff1 = sources[i]['Teff1']
#     meta = sources[i]['mh_for_mass_interp']
#     nuv_mag = sources[i]['nuv_mag']
#     fuv_mag = sources[i]['fuv_mag']

#     observed_nuv = 10**(-0.4 * (nuv_mag - nuv_m0)) * nuv_f0
#     observed_fuv = 10**(-0.4 * (fuv_mag - fuv_m0)) * fuv_f0

#     ms = model_sed_ms(teff1,m1,m1_err,parallax,parallax_err, meta)
#     wl_n = ms[ms['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
#     model_nuv = ms[ms['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / wl_n
#     model_nuv *= ext.extinguish(wl_n * u.AA ,Av=av)
#     n_excess.append(observed_nuv / model_nuv)

#     wl_f = ms[ms['filter'] == 'GALEX/GALEX.FUV']['wavelength'][0]
#     model_fuv = ms[ms['filter'] == 'GALEX/GALEX.FUV']['flux'][0] / wl_f
#     model_fuv *= ext.extinguish(wl_f * u.AA ,Av=av)
#     f_excess.append(observed_fuv / model_fuv)

# print(n_excess)
# print(f_excess)

[4.135635793343776, 0.5235170273851025, 2.334866859623821, 1.3938757595079896, 1.5231411681219156, masked]
[masked, masked, masked, masked, masked, 124.59903412691156]
